# OTROS TIEMPOS

In [1]:
weeks = 2

#52 semanas 8 minutos 
# 5 semanas 50 segundos

In [2]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)
# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

lost_sales_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.operarios.tiempos", "search",
    [[["inicio", ">=", FECHA_INICIO], ["inicio", "<=", FECHA_FIN]]]
)

lost_sales_data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.operarios.tiempos", "read",
    [lost_sales_ids],
    {"fields": []}
)



#"display_name","inicio","finalizo","is_pause"

df = pd.DataFrame(lost_sales_data)

df['tarea'] = df['tarea_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

#Tarda  menos de 1 minuto

📅 Descargando registros desde 2025-10-17 hasta 2025-10-31


In [3]:
df_ot = df[df["tipo"]!="sistema"] #Se quitan las de op ya que no caen bien

df_ot.columns = df_ot.columns.str.lower()
df_ot = df_ot.rename(columns={
    "display_name": "operario",
    "inicio": "fecha_inicio",
    "tipo": "actividad",
})

# Convertir fechas
for col in ["fecha_inicio", "create_date", "__last_update"]:
    df_ot[col] = pd.to_datetime(df_ot[col], errors="coerce")

# Agregar columnas que no existen en df_ot para igualar estructura
df_ot["duracion_horas"] = df_ot["delay_hours"]
df_ot["centro_produccion"] = "OTRAS TAREAS"

df_ot = df_ot[["operario","__last_update","actividad","finalizo","fecha_inicio","tarea","duracion_horas"]]

# Hora actual (en tu zona horaria si quieres)
ahora = pd.Timestamp.now()

df_ot["fecha_inicio"] = pd.to_datetime(df_ot["fecha_inicio"], errors="coerce")
df_ot["finalizo"] = pd.to_datetime(df_ot["finalizo"], errors="coerce")

# --- 1️⃣ Crear columna en_curso ---
# True si no hay fecha de finalización
df_ot["en_curso"] = df_ot["finalizo"].isna() | (df_ot["finalizo"] == False)

# --- 2️⃣ Recalcular duración ---
df_ot["duracion_horas"] = df_ot.apply(
    lambda row: (
        (ahora - row["fecha_inicio"]).total_seconds() / 3600
        if row["en_curso"] and pd.notna(row["fecha_inicio"])
        else (row["finalizo"] - row["fecha_inicio"]).total_seconds() / 3600
        if pd.notna(row["finalizo"]) and pd.notna(row["fecha_inicio"])
        else 0
    ),
    axis=1
).round(2)

# TIEMPOS POR OP

In [4]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd
import re

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)
# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

ops_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.tiempos.trabajo", "search",
    [[["create_date", ">=", FECHA_INICIO], ["create_date", "<=", FECHA_FIN]]]
)

t_op = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.tiempos.trabajo", "read",
    [ops_ids],
    {"fields": []}
)



#"display_name","inicio","finalizo","is_pause"

df_op = pd.DataFrame(t_op)

df_op['Display_name'] = df_op['operario'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

df_op['OP'] = df_op['production_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

df_op['centro_produccion'] = df_op['workcenter_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))


df_op["nom_filtro"] = df_op["product_fabricado_id"].astype(str).apply(
    lambda x: re.search(r"\(([^)]+)\)", x).group(1) if re.search(r"\(([^)]+)\)", x) else None
)

df_op["product_ref"] = df_op["product_fabricado_id"].astype(str).apply(
    lambda x: re.search(r"\[([A-Z0-9]+)\]", x).group(1) if re.search(r"\[([A-Z0-9]+)\]", x) else None
)

df_op["maquina"] = df_op["fuerza_lab_id"].astype(str).apply(
    lambda x: re.search(r",\s*[\'\"]?([A-ZÁÉÍÓÚÑ0-9\s]+?)(?=\s*-\s*)", x, re.IGNORECASE).group(1).strip()
    if re.search(r",\s*[\'\"]?([A-ZÁÉÍÓÚÑ0-9\s]+?)(?=\s*-\s*)", x, re.IGNORECASE)
    else None
)



df_op = df_op.drop(columns=["operario", "production_id", "workcenter_id","write_uid","id","display_name","create_uid","wc_line_id","product_fabricado_id"])

#Tarda aprox 4 minutos


📅 Descargando registros desde 2025-10-17 hasta 2025-10-31


In [5]:
# --- Limpieza df_op ---
df_op.columns = df_op.columns.str.lower()
df_op = df_op.rename(columns={"evento": "actividad", "display_name": "operario"})

# Asegurar tipos
for col in ["fecha_evento", "create_date", "write_date", "__last_update"]:
    df_op[col] = pd.to_datetime(df_op[col], errors="coerce")


In [6]:
from datetime import timedelta

# Asegurar tipos de fecha correctos
df_op["fecha_evento"] = pd.to_datetime(df_op["fecha_evento"], errors="coerce")

# --- 1️⃣ Crear columna 'final' ---
df_op["final"] = df_op.apply(
    lambda row: row["fecha_evento"] + timedelta(seconds=row["segundos_evento"])
    if pd.notna(row["segundos_evento"]) else row["fecha_evento"],
    axis=1
)

# --- 2️⃣ Ordenar por operario y fecha_evento ---
df_op = df_op.sort_values(["operario", "fecha_evento"])

# --- 3️⃣ Marcar el último evento por operario ---
df_op["ultimo_evento"] = df_op.groupby("operario")["fecha_evento"].transform("max")

# --- 4️⃣ Crear columna en_curso ---
# En curso = el evento más reciente del operario y su actividad es "iniciar"
df_op["en_curso"] = (df_op["fecha_evento"] == df_op["ultimo_evento"]) & (df_op["actividad"] == "iniciar")

# --- 5️⃣ Filtrar: eliminar 'iniciar' antiguos (solo mantener el último 'iniciar' si está en curso) ---
df_op_filtrado = df_op[
    (df_op["actividad"] != "iniciar") |  # mantener todo lo que no sea iniciar
    (df_op["en_curso"])                  # o mantener el último iniciar de cada operario
].copy()

# --- 6️⃣ Limpieza final ---
df_op_filtrado = df_op_filtrado.drop(columns=["ultimo_evento"])


## UNIR DATAFRAMES

In [7]:
# Copias de trabajo
df1 = df_op_filtrado.copy()
df2 = df_ot.copy()

# --- Homologar df_op_filtrado a estructura de df_ot ---
df1 = df1.rename(columns={
    'fecha_evento': 'fecha_inicio',
    'final': 'finalizo',
    'horas_evento': 'duracion_horas',
    'centro_produccion': 'tarea'
})

# --- Agregar columna de origen ---
df1['origen'] = 'produccion'
df2['origen'] = 'otras_tareas'

# --- Crear columna de segundos_evento ---
# En producción ya existe, pero renombramos para coherencia
if 'segundos_evento' not in df1.columns:
    df1['segundos_evento'] = df1['duracion_horas'] * 3600
# En tareas la calculamos desde duracion_horas
df2['segundos_evento'] = df2['duracion_horas'] * 3600

# --- Unir ambos sin perder datos ---
df_union = pd.concat([df1, df2], ignore_index=True, sort=False)

# --- Ordenar columnas ---
cols_base = [
    'operario', '__last_update', 'actividad', 'fecha_inicio', 'finalizo',
    'tarea', 'duracion_horas', 'segundos_evento', 'en_curso', 'origen'
]

cols_extra = [
    'op', 'fuerza_lab_id', 'maquina', 'nom_filtro', 'product_ref',
    'create_date', 'write_date'
]

cols_final = cols_base + [c for c in cols_extra if c in df_union.columns]
df_union = df_union[cols_final]


if 'tarea' in df_union.columns:
    df_union['tarea'] = df_union['tarea'].astype(str).str.upper()


# Tomar fecha y hora actual
ahora = pd.Timestamp.now()

# Filtrar filas en curso
mask_en_curso = df_union['en_curso'] == True

# Calcular diferencia de tiempo
df_union.loc[mask_en_curso, 'duracion_horas'] = (
    (ahora - df_union.loc[mask_en_curso, 'fecha_inicio']).dt.total_seconds() / 3600
)

df_union.loc[mask_en_curso, 'segundos_evento'] = (
    df_union.loc[mask_en_curso, 'duracion_horas'] * 3600
)

# Asegurar que la columna sea texto y eliminar espacios
df_union['op'] = df_union['op'].astype(str).str.strip()

df_union['operario'] = df_union['operario'].astype(str).str.strip()

df_union.loc[df_union['actividad'].str.lower().str.strip() == 'produccion', 'actividad'] = 'otras_tareas'

# Guardar a Excel
df_union.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/operarios_tiempo_completo.xlsx", index=False)

#TARDA 1 MINUTO


In [8]:
df_union[df_union["en_curso"]==True].head()

,operario,__last_update,actividad,fecha_inicio,finalizo,tarea,duracion_horas,segundos_evento,en_curso,origen,op,fuerza_lab_id,maquina,nom_filtro,product_ref,create_date,write_date


In [9]:
#df_union[(df_union["operario"]=="JAMER PUPO SIERRA")& (df_union["fecha_inicio"]>"2025-10-21")].head(7)